## Introduction

In this chapter, you will understand how Spark represents and manages tables (or tabular data). Different programming languages and frameworks use different names to describe a table. But, in Apache Spark, tables are referred as Spark DataFrames.

In `pyspark`, these DataFrames are stored inside python objects of class `pyspark.sql.dataframe.DataFrame`, and all the methods present in this class, are commonly referred as the DataFrame API of Spark. This is the most important API of Spark. Much of your Spark applications will heavily use this API to compose your data transformations and data flows [@chambers2018].

## Spark DataFrames versus Spark Datasets

Spark have two notions of structured data: DataFrames and Datasets. In summary, a Spark Dataset, is a distributed collection of data [@sparkdoc]. In contrast, a Spark DataFrame is a Spark Dataset organized into named columns [@sparkdoc].

This means that, Spark DataFrames are very similar to tables as we know in relational databases - RDBMS. So in a Spark DataFrame, each column has a name, and they all have the same number of rows. Furthermore, all the rows inside a column must store the same type of data, but each column can store a different type of data.

In the other hand, Spark Datasets are considered a collection of any type of data. So a Dataset might be a collection of unstructured data as well, like log files, JSON and XML trees, etc. Spark Datasets can be created and transformed trough the Dataset API of Spark. But this API is available only in Scala and Java API's of Spark. For this reason, we do not act directly on Datasets with `pyspark`, only DataFrames. That's ok, because for the most part of applications, we do want to use DataFrames, and not Datasets, to represent our data.

However, what makes a Spark DataFrame different from other dataframes? Like the `pandas` DataFrame? Or the R native `data.frame` structure? Is the **distributed** aspect of it. Spark DataFrames are based on Spark Datasets, and these Datasets are collections of data that are distributed across the cluster. As an example, lets suppose you have the following table stored as a Spark DataFrame:

| ID  | Name   | Value |
|-----|--------|-------|
| 1   | Anne   | 502   |
| 2   | Carls  | 432   |
| 3   | Stoll  | 444   |
| 4   | Percy  | 963   |
| 5   | Martha | 123   |
| 6   | Sigrid | 621   |

If you are running Spark in a 4 nodes cluster (one is the driver node, and the other three are worker nodes). Each worker node of the cluster will store a section of this data. So you, as the programmer, will see, manage and transform this table as if it was a single and unified table. But behind the hoods, Spark will split this data and store it as many fragments across the Spark cluster. @fig-distributed-df presents this notion in a visual manner.

![A Spark DataFrame is distributed across the cluster](../Figures/distributed-df.png){#fig-distributed-df fig-align="center"}


## Building a Spark DataFrame

There are some different methods to create a Spark DataFrame. For example, because a DataFrame is basically a Dataset of rows, we can build a DataFrame from a collection of `Row`'s, through the `createDataFrame()` method from your Spark Session:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from datetime import date
from pyspark.sql import Row

data = [
  Row(id = 1, value = 28.3, date = date(2021,1,1)),
  Row(id = 2, value = 15.8, date = date(2021,1,1)),
  Row(id = 3, value = 20.1, date = date(2021,1,2)),
  Row(id = 4, value = 12.6, date = date(2021,1,3))
]

df = spark.createDataFrame(data)

Remember that a Spark DataFrame in python is a object of class `pyspark.sql.dataframe.DataFrame` as you can see below. If you try to see what is inside of this kind of object, you will get a small description of the columns present in the DataFrame as a result:

In [ ]:
#| eval: false
type(df)

```{verbatim}
<class 'pyspark.sql.dataframe.DataFrame'>
```


In [ ]:
df

So, in the above example, we use the `Row()` constructor (from `pyspark.sql` module) to build 4 rows. The `createDataFrame()` method, stack these 4 rows together to form our new DataFrame `df`. The result is a Spark DataFrame with 4 rows and 3 columns (`id`, `value` and `date`).

But you can use different methods to create the same Spark DataFrame. As another example, with the code below, we are creating a DataFrame called `students`. To do this, we create two python lists (`data` and `columns`), then, deliver these lists to `createDataFrame()` method. Each element of `data` is a python `tuple` that represents a row in the `students` DataFrame. And each element of `columns` represent the name of a column present in this new DataFrame.

In [ ]:
data = [
  (12114, 'Anne', 21, 1.56, 8, 9, 10, 9, 'Economics', 'SC'),
  (13007, 'Adrian', 23, 1.82, 6, 6, 8, 7, 'Economics', 'SC'),
  (10045, 'George', 29, 1.77, 10, 9, 10, 7, 'Law', 'SC'),
  (12459, 'Adeline', 26, 1.61, 8, 6, 7, 7, 'Law', 'SC'),
  (10190, 'Mayla', 22, 1.67, 7, 7, 7, 9, 'Design', 'AR'),
  (11552, 'Daniel', 24, 1.75, 9, 9, 10, 9, 'Design', 'AR')
]

columns = [
  'StudentID', 'Name', 'Age', 'Height', 'Score1',
  'Score2', 'Score3', 'Score4', 'Course', 'Department'
]

students = spark.createDataFrame(data, columns)
students

If you need to, you can easily collect a python list with the column names present in your DataFrame, in the same way you would do in a `pandas` DataFrame. That is, by using the `columns` method of your DataFrame, like this:

In [ ]:
students.columns

## Viewing a Spark DataFrame

A key aspect of Spark is its laziness. In other words, for most operations, Spark will only check if your code is correct and if it makes sense. Spark will not actually run or execute the operations you are describing in your code, unless you explicit ask for it with a trigger operation, that is called an "action" (this kind of operation is described in @sec-dataframe-actions).

You can notice this laziness in the above output. Because when we call for an object that stores a Spark DataFrame (like `df` and `students`), Spark will only calculate and print a summary of the structure of your Spark DataFrame, and not the DataFrame itself.

So how can we actually see our DataFrame? How can we visualize the rows and values that are stored inside of it? We use the `show()` method. With this method, Spark will print the table as pure text, as you can see in the example below:

In [ ]:
students.show()

By default, this method shows only the top rows of your DataFrame, but you can specify how much rows exactly you want to see, by using `show(n)`, where `n` is the number of rows. For example, I can visualize only the first 2 rows of `df` like this:

In [ ]:
df.show(2)

## The DataFrame Schema

The schema of a Spark DataFrame is the combination of column names and the data types associated with each of these columns. Schemas can be set explicitly by you (that is, you can tell Spark how the schema of your DataFrame should look like), or, can be defined automatically by Spark while reading and creating data. You can get a succinct description of a DataFrame schema, by looking inside the object where this DataFrame is stored. 

For example, lets look again to the `df` DataFrame. In the result below, we can see that `df` has three columns (`id`, `value` and `date`). By the description `id: bigint`, we know that `id` is a column of type `bigint`, which translates to the `LongType()` of Spark (we will talk shortly about these data types). Furthermore, by the descriptions `value: double` and `date: date`, we know too that the columns `value` and `date` are of type `double` and `date`, respectively.

In [ ]:
df

So, by calling the object of your DataFrame (i.e. an object of class `pyspark.sql.dataframe.DataFrame`) in the console, you can see a small description of the schema of this DataFrame. But, how can you access the description of this schema programmatically? You do this, by using the `schema` method of your DataFrame, like in the example below:

In [ ]:
df.schema

The result of the `schema` method, is a `StructType()` object, that contains some information about each column of your DataFrame. More specifically, a `StructType()` object is filled with multiple `StructField()` objects. Each `StructField()` object store the name and the type of a column, and a boolean value (`True` or `False`) that indicates if this column can contain any null value inside of it.

You can use a `for` loop to iterate through this `StructType()` and get the information of each column separately.

In [ ]:
schema = df.schema
for column in schema:
  print(column)

You can access just the data type of each column by using the `dataType` method of each `StructField()` object.

In [ ]:
for column in schema:
  datatype = column.dataType
  print(datatype)

And you can do the same for column names and the boolean value (that indicates if the column can contain "null" values), by using the `name` and `nullable` methods, respectively.

In [ ]:
# Accessing the name of each column
for column in schema:
  print(column.name)

In [ ]:
# Accessing the boolean value that indicates
# if the column can contain null values
for column in schema:
  print(column.nullable)

## Partitions of a Spark DataFrame

As we exposed in @fig-distributed-df, a Spark DataFrame is broken into many small parts that are distributed across the cluster. Each one of these parts are considered a DataFrame *partition*.
